In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "notebooks"
    cd(joinpath(@__DIR__, "../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Desktop/Doktor/MorphoMol/MorphoMolMonteCarlo`


In [3]:
folder = "assets/hpc_assembly/3_6r7m_assembled_oj_0_0_os_0_85/"
#folder = "assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, "_")[1]) ? parse(Int, split(file, "_")[1]) : max
    end
end

evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        @load "$folder$file" in_out_data
        id = parse(Int, split(file, "_")[1])
        string = "$(id): $(in_out_data.input.T) | $(in_out_data.input.ε) | $(in_out_data.output.αs[end]) | $(in_out_data.output.Es[1]), $(minimum(in_out_data.output.Es)), $(in_out_data.output.Es[end])"
        evaluation_strings[id] = string
    end
end

In [4]:
for elem in evaluation_strings
    println(elem)
end











11: 2.0 | 0.001 | 0.76650804 | 12169.566358056172, 12167.2173633987, 12249.712112751124
12: 2.0 | 0.001 | 0.7467844 | 12169.566358056172, 12167.032218678713, 12291.353717390131
13: 2.0 | 0.001 | 0.80403346 | 12169.566358056172, 12166.689486914558, 12225.037356925466
14: 2.0 | 0.001 | 0.7905313 | 12169.566358056172, 12167.70688242698, 12225.83411195816
15: 2.0 | 0.001 | 0.7466495 | 12169.566358056172, 12167.918953802699, 12285.065525008342
16: 2.0 | 0.001 | 0.75485474 | 12169.566358056172, 12167.982937889921, 12214.916185598064
17: 2.0 | 0.001 | 0.72323173 | 12169.566358056172, 12166.040715064779, 12238.46759809368
18: 2.0 | 0.001 | 0.7835406 | 12169.566358056172, 12165.106618405653, 12226.640088009704
19: 2.0 | 0.001 | 0.77945644 | 12169.566358056172, 12167.209035919193, 12237.791491360264
20: 2.0 | 0.001 | 0.7813851 | 12169.566358056172, 12166.757142373854, 12251.742331865033
21: 2.0 | 0.003 | 0.32538933 | 12169.566358056172, 12169.566358056172, 12221.394554937302
22: 2.0 | 

In [4]:
function get_flat_realization(x, template_mol)
    n_mol = length(x) ÷ 6
    [(hvcat((n_mol), [exp(Rotations.RotationVecGenerator(x[i:i+2]...)) * template_mol .+ x[i+3:i+5] for i in 1:6:length(x)]...)...)...]
end

get_flat_realization (generic function with 1 method)

In [45]:
folder = "assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/"

for file in readdir(folder)
    try
        @load "$folder$file" in_out_data
        println("$folder$file")
        mindex = argmin(in_out_data.output.Es)
        state = in_out_data.output.states[mindex]
        id = parse(Int, split(file, "_")[1])
        radii = Base.Iterators.collect(Base.Iterators.flatten([in_out_data.input.template_radii for _ in 1:in_out_data.input.number_of_molecules]))
        MorphoMol.Utilities.state_to_poly(get_flat_realization(state, in_out_data.input.template_mol), radii, "$(folder)$(id)", in_out_data.input.number_of_molecules, length(in_out_data.input.template_radii))
    catch
        continue
    end
end

assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/86_6r7m_protor_oj_0_0_os_0_85.jld2
assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/87_6r7m_protor_oj_0_0_os_0_85.jld2
assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/88_6r7m_protor_oj_0_0_os_0_85.jld2
assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/89_6r7m_protor_oj_0_0_os_0_85.jld2
assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/90_6r7m_protor_oj_0_0_os_0_85.jld2


In [55]:
function get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, subset_a::Vector{Int}, subset_b::Vector{Int})
    function transform_dist(template_centers, state, ss, idx)
        i = ss[1]
        j = ss[2]
        R1 = exp(RotationVecGenerator(state[(i-1) * 6 + 1:(i-1) * 6 + 3]...))
        T1 = state[(i-1) * 6 + 4:(i-1) * 6 + 6]
        R2 = exp(RotationVecGenerator(state[(j-1) * 6 + 1:(j-1) * 6 + 3]...))
        T2 = state[(j-1) * 6 + 4:(j-1) * 6 + 6]
        euclidean(T1 + R1 * template_centers[:,idx], T2 + R2 * template_centers[:,idx])
    end

    [abs(transform_dist(template_centers_a, state_a, subset_a, i) - transform_dist(template_centers_b, state_b, subset_b, i)) for i in 1:size(template_centers)[2]]
end

function sum_of_permutation(template_centers_a, template_centers_b, state_a, state_b, perm_a::Vector{Int}, perm_b::Vector{Int})
    n = size(template_centers)[2]
    d1 = sum(get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, perm_a[[1,2]], perm_b[[1,2]])) / n
    d2 = sum(get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, perm_a[[2,3]], perm_b[[2,3]])) / n
    d3 = sum(get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, perm_a[[1,3]], perm_b[[1,3]])) / n
    println("$(d1), $(d2), $(d3)")
    (d1 + d2 + d3)/3.0
end

sum_of_permutation (generic function with 2 methods)

In [57]:
@load "assets/hpc_assembly/3_6r7m_assembled_oj_0_0_os_0_85/49_6r7m_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

template_mol_exp, _ = MorphoMol.Utilities.poly_to_state("assets/input/poly/6r7m_protor.poly")
template_mol_exp = reshape(template_mol_exp,(3,Int(length(template_mol_exp) / 3)))

R1 = log(RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000]))
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = log(RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000]))
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = log(RotMatrix(@SMatrix[0.992567  0.121696  0.000000; -0.121696  0.992567  0.000000; 0.000000  0.000000  1.000000]))
T3 = @SVector[-19.48193, 22.01644, -46.53000]

state_exp_assembled = Vector{Float64}([])
for (R,T) in [(R1, T1), (R2, T2), (R3, T3)]
    state_exp_assembled = [state_exp_assembled; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end

permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(sum_of_permutation(template_mol_exp, template_mol_sim, state_exp_assembled, state_sim, [1, 2, 3], perm))
end

1.250584714175155, 18.68099252243644, 22.633764326901876
14.188447187837824
1.2639753574258612, 18.68099252243644, 22.6202347276333
14.1884008691652
1.250584714175155, 1.2640841390049435, 2.688687665460494
1.7344521728801976
18.68110130401552, 1.2640841390049435, 22.6202347276333
14.188473390217922
1.2639753574258612, 1.2506931265693904, 2.688687665460494
1.7344520498185816
18.68110130401552, 1.2506931265693904, 22.633764326901876
14.188519585828928


In [58]:
@load "assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/43_6r7m_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

template_mol_exp, _ = MorphoMol.Utilities.poly_to_state("assets/input/poly/6r7m_protor.poly")
template_mol_exp = reshape(template_mol_exp,(3,Int(length(template_mol_exp) / 3)))

R1 = log(RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000]))
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = log(RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000]))
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = log(RotMatrix(@SMatrix[0.803441  0.595384  0.000000; -0.595384  0.803441  0.000000; 0.000000  0.000000  1.000000]))
T3 = @SVector[-67.99970, 135.02619, -25.38000]


state_exp_assembled = Vector{Float64}([])
for (R,T) in [(R1, T1), (R2, T2), (R3, T3)]
    state_exp_assembled = [state_exp_assembled; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end

permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(sum_of_permutation(template_mol_exp, template_mol_sim, state_exp_assembled, state_sim, [1, 2, 3], perm))
    println()
end

1.060546051243098, 1.7233861453834445, 2.704512138639089
1.829481445088544

3.8582220739511452, 1.7233861453834445, 7.206604684868613
4.262737634734401

1.060546051243098, 1.9544367320546299, 6.025618439072101
3.0135337407899434

5.358151778229962, 1.9544367320546299, 7.206604684868613
4.839731065051068

3.8582220739511452, 5.183991323542944, 6.025618439072101
5.02261061218873

5.358151778229962, 5.183991323542944, 2.704512138639089
4.415551746803998

